In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification, make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully!")


In [ ]:
class DetailedNeuralNetwork:
    """Neural Network with detailed forward/backward propagation logging"""
    
    def __init__(self, layer_sizes, activation='sigmoid', learning_rate=0.01):
        self.layer_sizes = layer_sizes
        self.activation = activation
        self.learning_rate = learning_rate
        self.num_layers = len(layer_sizes)
        
        # Initialize weights and biases
        self.weights = {}
        self.biases = {}
        
        for i in range(1, self.num_layers):
            # Xavier initialization
            self.weights[i] = np.random.randn(layer_sizes[i-1], layer_sizes[i]) * np.sqrt(2.0 / layer_sizes[i-1])
            self.biases[i] = np.zeros((1, layer_sizes[i]))
        
        # Storage for forward pass values
        self.z_values = {}  # Pre-activation values
        self.a_values = {}  # Post-activation values
        
        # Storage for backward pass values
        self.dW = {}  # Weight gradients
        self.db = {}  # Bias gradients
        self.dz = {}  # Pre-activation gradients
        self.da = {}  # Post-activation gradients
        
        self.training_history = []
        
    def sigmoid(self, x):
        """Sigmoid activation with numerical stability"""
        x = np.clip(x, -500, 500)
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_derivative(self, x):
        """Derivative of sigmoid"""
        s = self.sigmoid(x)
        return s * (1 - s)
    
    def relu(self, x):
        """ReLU activation"""
        return np.maximum(0, x)
    
    def relu_derivative(self, x):
        """Derivative of ReLU"""
        return (x > 0).astype(float)
    
    def apply_activation(self, z):
        """Apply chosen activation function"""
        if self.activation == 'sigmoid':
            return self.sigmoid(z)
        elif self.activation == 'relu':
            return self.relu(z)
        else:
            raise ValueError(f"Unknown activation: {self.activation}")
    
    def apply_activation_derivative(self, z):
        """Apply activation derivative"""
        if self.activation == 'sigmoid':
            return self.sigmoid_derivative(z)
        elif self.activation == 'relu':
            return self.relu_derivative(z)
        else:
            raise ValueError(f"Unknown activation: {self.activation}")
    
    def forward_propagation(self, X, verbose=False):
        """Detailed forward propagation with logging"""
        self.a_values[0] = X  # Input layer
        
        if verbose:
            print("\n=== FORWARD PROPAGATION ===")
            print(f"Input (a[0]): shape {X.shape}")
            print(f"Sample values: {X[0][:3] if len(X[0]) > 3 else X[0]}")\n        \n        for layer in range(1, self.num_layers):\n            # Linear transformation: z = W*a + b\n            self.z_values[layer] = np.dot(self.a_values[layer-1], self.weights[layer]) + self.biases[layer]\n            \n            # Apply activation function: a = σ(z)\n            if layer == self.num_layers - 1:  # Output layer\n                self.a_values[layer] = self.sigmoid(self.z_values[layer])  # Always sigmoid for binary classification\n            else:  # Hidden layers\n                self.a_values[layer] = self.apply_activation(self.z_values[layer])\n            \n            if verbose:\n                print(f\"\\nLayer {layer}:\")\n                print(f\"  Weights shape: {self.weights[layer].shape}\")\n                print(f\"  Pre-activation (z[{layer}]) shape: {self.z_values[layer].shape}\")\n                print(f\"  Pre-activation sample: {self.z_values[layer][0][:3] if len(self.z_values[layer][0]) > 3 else self.z_values[layer][0]}\")\n                print(f\"  Post-activation (a[{layer}]) shape: {self.a_values[layer].shape}\")\n                print(f\"  Post-activation sample: {self.a_values[layer][0][:3] if len(self.a_values[layer][0]) > 3 else self.a_values[layer][0]}\")\n        \n        return self.a_values[self.num_layers - 1]\n    \n    def compute_cost(self, y_true, y_pred):\n        \"\"\"Binary cross-entropy cost function\"\"\"\n        m = y_true.shape[0]\n        cost = -np.mean(y_true * np.log(y_pred + 1e-15) + (1 - y_true) * np.log(1 - y_pred + 1e-15))\n        return cost\n    \n    def backward_propagation(self, X, y, verbose=False):\n        \"\"\"Detailed backward propagation with logging\"\"\"\n        m = X.shape[0]\n        y = y.reshape(-1, 1)\n        \n        if verbose:\n            print(\"\\n=== BACKWARD PROPAGATION ===\")\n        \n        # Output layer gradients\n        output_layer = self.num_layers - 1\n        \n        # dC/da (cost gradient w.r.t. final activation)\n        self.da[output_layer] = -(y / self.a_values[output_layer] - (1 - y) / (1 - self.a_values[output_layer]))\n        \n        # dC/dz (cost gradient w.r.t. final pre-activation)\n        # For sigmoid: da/dz = a(1-a)\n        self.dz[output_layer] = self.da[output_layer] * self.a_values[output_layer] * (1 - self.a_values[output_layer])\n        \n        # Simplified for binary classification: dC/dz = a - y\n        self.dz[output_layer] = self.a_values[output_layer] - y\n        \n        if verbose:\n            print(f\"\\nOutput Layer {output_layer}:\")\n            print(f\"  da[{output_layer}] shape: {self.da[output_layer].shape}\")\n            print(f\"  dz[{output_layer}] shape: {self.dz[output_layer].shape}\")\n            print(f\"  dz[{output_layer}] sample: {self.dz[output_layer][0]}\")\n        \n        # Propagate gradients backward through all layers\n        for layer in range(output_layer, 0, -1):\n            # Gradients w.r.t. weights and biases\n            self.dW[layer] = (1/m) * np.dot(self.a_values[layer-1].T, self.dz[layer])\n            self.db[layer] = (1/m) * np.sum(self.dz[layer], axis=0, keepdims=True)\n            \n            if verbose:\n                print(f\"\\nLayer {layer} gradients:\")\n                print(f\"  dW[{layer}] shape: {self.dW[layer].shape}\")\n                print(f\"  db[{layer}] shape: {self.db[layer].shape}\")\n                print(f\"  dW[{layer}] sample: {self.dW[layer][:2, :2] if self.dW[layer].shape[0] > 2 else self.dW[layer]}\")\n            \n            # Propagate error to previous layer (if not input layer)\n            if layer > 1:\n                # da = W^T * dz (from next layer)\n                self.da[layer-1] = np.dot(self.dz[layer], self.weights[layer].T)\n                \n                # dz = da * σ'(z)\n                self.dz[layer-1] = self.da[layer-1] * self.apply_activation_derivative(self.z_values[layer-1])\n                \n                if verbose:\n                    print(f\"  Propagated da[{layer-1}] shape: {self.da[layer-1].shape}\")\n                    print(f\"  Propagated dz[{layer-1}] shape: {self.dz[layer-1].shape}\")\n    \n    def update_parameters(self, verbose=False):\n        \"\"\"Update weights and biases using gradients\"\"\"\n        if verbose:\n            print(\"\\n=== PARAMETER UPDATE ===\")\n        \n        for layer in range(1, self.num_layers):\n            old_w = self.weights[layer].copy()\n            old_b = self.biases[layer].copy()\n            \n            # Update weights and biases\n            self.weights[layer] -= self.learning_rate * self.dW[layer]\n            self.biases[layer] -= self.learning_rate * self.db[layer]\n            \n            if verbose:\n                weight_change = np.mean(np.abs(self.weights[layer] - old_w))\n                bias_change = np.mean(np.abs(self.biases[layer] - old_b))\n                print(f\"Layer {layer}: Avg weight change: {weight_change:.6f}, Avg bias change: {bias_change:.6f}\")\n    \n    def train_one_epoch(self, X, y, verbose=False):\n        \"\"\"Train for one epoch with detailed logging\"\"\"\n        # Forward pass\n        predictions = self.forward_propagation(X, verbose=verbose)\n        \n        # Compute cost\n        cost = self.compute_cost(y.reshape(-1, 1), predictions)\n        \n        # Backward pass\n        self.backward_propagation(X, y, verbose=verbose)\n        \n        # Update parameters\n        self.update_parameters(verbose=verbose)\n        \n        return cost, predictions\n\n# Test with a simple example\nprint(\"=== Creating Simple Dataset for Detailed Analysis ===\")\n\n# Create simple dataset\nnp.random.seed(42)\nX_simple = np.array([\n    [0.1, 0.2],\n    [0.8, 0.9],\n    [0.2, 0.8],\n    [0.9, 0.1]\n])\ny_simple = np.array([0, 1, 1, 0])  # XOR-like pattern\n\nprint(f\"Input data shape: {X_simple.shape}\")\nprint(f\"Input data:\\n{X_simple}\")\nprint(f\"Labels: {y_simple}\")\n\n# Create small network for detailed analysis\ndetailed_nn = DetailedNeuralNetwork(\n    layer_sizes=[2, 3, 1],  # 2 inputs -> 3 hidden -> 1 output\n    activation='sigmoid',\n    learning_rate=1.0\n)\n\nprint(f\"\\nNetwork architecture: {detailed_nn.layer_sizes}\")\nprint(f\"Number of parameters:\")\nfor layer in range(1, detailed_nn.num_layers):\n    n_weights = detailed_nn.weights[layer].size\n    n_biases = detailed_nn.biases[layer].size\n    print(f\"  Layer {layer}: {n_weights} weights + {n_biases} biases = {n_weights + n_biases} total\")\n\ntotal_params = sum(w.size for w in detailed_nn.weights.values()) + sum(b.size for b in detailed_nn.biases.values())\nprint(f\"Total parameters: {total_params}\")


In [ ]:
# Demonstrate one detailed training step
print("\n" + "="*80)
print("DETAILED TRAINING STEP ANALYSIS")
print("="*80)

# Run one training epoch with full verbosity
cost, predictions = detailed_nn.train_one_epoch(X_simple, y_simple, verbose=True)

print(f"\n=== TRAINING RESULTS ===")
print(f"Cost: {cost:.6f}")
print(f"Predictions: {predictions.flatten()}")
print(f"True labels: {y_simple}")
print(f"Accuracy: {accuracy_score(y_simple, (predictions > 0.5).astype(int)):.2f}")


In [ ]:
# Function to analyze gradient magnitudes
def analyze_gradients(network, X, y, num_epochs=100):
    """Analyze gradient magnitudes throughout training"""
    gradient_history = {}
    cost_history = []\n    \n    for layer in range(1, network.num_layers):\n        gradient_history[f'dW_{layer}'] = []\n        gradient_history[f'db_{layer}'] = []\n    \n    for epoch in range(num_epochs):\n        cost, _ = network.train_one_epoch(X, y, verbose=False)\n        cost_history.append(cost)\n        \n        # Record gradient magnitudes\n        for layer in range(1, network.num_layers):\n            gradient_history[f'dW_{layer}'].append(np.mean(np.abs(network.dW[layer])))\n            gradient_history[f'db_{layer}'].append(np.mean(np.abs(network.db[layer])))\n    \n    return gradient_history, cost_history\n\n# Test gradient flow with different network depths\ndepths = [2, 4, 6]  # Different numbers of hidden layers\nactivations = ['sigmoid', 'relu']\n\n# Create a more complex dataset\nX_complex, y_complex = make_moons(n_samples=200, noise=0.1, random_state=42)\nscaler = StandardScaler()\nX_complex = scaler.fit_transform(X_complex)\n\nfig, axes = plt.subplots(len(activations), len(depths), figsize=(18, 12))\n\nfor i, activation in enumerate(activations):\n    for j, depth in enumerate(depths):\n        # Create network with varying depth\n        layer_sizes = [2] + [10] * depth + [1]\n        \n        network = DetailedNeuralNetwork(\n            layer_sizes=layer_sizes,\n            activation=activation,\n            learning_rate=0.01\n        )\n        \n        # Analyze gradients\n        grad_history, cost_history = analyze_gradients(network, X_complex, y_complex, num_epochs=50)\n        \n        # Plot gradient magnitudes\n        for layer in range(1, len(layer_sizes)):\n            if layer < len(layer_sizes) - 1:  # Hidden layers\n                axes[i, j].plot(grad_history[f'dW_{layer}'], \n                               label=f'Hidden Layer {layer}', alpha=0.7)\n            else:  # Output layer\n                axes[i, j].plot(grad_history[f'dW_{layer}'], \n                               label=f'Output Layer', linewidth=2)\n        \n        axes[i, j].set_title(f'{activation.capitalize()}, {depth} Hidden Layers')\n        axes[i, j].set_xlabel('Epoch')\n        axes[i, j].set_ylabel('Average Gradient Magnitude')\n        axes[i, j].set_yscale('log')\n        axes[i, j].legend()\n        axes[i, j].grid(True, alpha=0.3)\n\nplt.suptitle('Gradient Flow Analysis: Effect of Depth and Activation Function', fontsize=16)\nplt.tight_layout()\nplt.show()\n\n# Demonstrate vanishing gradient problem\nprint(\"\\n=== VANISHING GRADIENT ANALYSIS ===\")\n\n# Deep network with sigmoid\ndeep_sigmoid = DetailedNeuralNetwork(\n    layer_sizes=[2, 10, 10, 10, 10, 10, 1],\n    activation='sigmoid',\n    learning_rate=0.01\n)\n\n# Train for a few epochs and check gradient magnitudes\nfor epoch in range(5):\n    cost, _ = deep_sigmoid.train_one_epoch(X_complex, y_complex)\n    \n    print(f\"\\nEpoch {epoch + 1}:\")\n    for layer in range(1, deep_sigmoid.num_layers):\n        grad_mag = np.mean(np.abs(deep_sigmoid.dW[layer]))\n        print(f\"  Layer {layer} gradient magnitude: {grad_mag:.8f}\")\n\nprint(\"\\nNote: In deep sigmoid networks, gradients often become very small in early layers!\")\nprint(\"This is the vanishing gradient problem that motivated ReLU and other innovations.\")"
